# Unit 3 Testing and Running ML Pipelines with Airflow CLI

Based on the previous conversation, here is the information converted to English:

Welcome to the third lesson in our "Automating Retraining with Apache Airflow" course.

In our previous lessons, we discussed the core concepts of Apache Airflow and how to design an ML pipeline DAG using the TaskFlow API. Now, let's learn how to test and run that pipeline effectively.

-----

## Understanding the Airflow CLI

The **Airflow Command Line Interface (CLI)** is a tool that allows you to interact with Airflow directly from the terminal. The CLI is critical for your ML pipeline development workflow for several reasons:

  * **Development and debugging**: You can quickly test DAGs and tasks during development without waiting for a scheduled run.
  * **CI/CD pipelines**: You can automate DAG validation in continuous integration workflows.
  * **Troubleshooting**: You can investigate issues in your workflows with detailed logging.

Using the CLI to test each component of your pipeline will help you catch issues early, such as a model trained on corrupted data or with incorrect hyperparameters, which could affect business decisions.

-----

## Exploring Available DAGs

Before you can test a DAG, you need to know what's available in your environment.

To list all available DAGs, run the following command:

```bash
airflow dags list
```

This command will display a tabular list of all DAGs, including the DAG ID, schedule interval, and whether each DAG is paused. Remember that the DAG code must be saved correctly in `$AIRFLOW_HOME/dags` to be discovered.

To examine the details of your ML pipeline DAG, use the command:

```bash
airflow dags show mlops_pipeline
```

This command reveals the architecture of your DAG, showing all tasks and their connections, allowing you to verify its structure before execution.

-----

## Testing Complete DAGs

Once you have a view of the architecture, you can test if the DAG runs correctly from start to finish. Use this command for a test run:

```bash
airflow dags test mlops_pipeline
```

When you run this command, your entire pipeline will be executed in sequence, from data extraction to model deployment. However, this test execution has a few unique characteristics:

  * It runs immediately instead of waiting for the next scheduled interval.
  * It shows all logs directly in your console.
  * It doesn't record the run in Airflow's database, keeping your metadata clean.

This is a safe way to observe every step without affecting your production environment.

-----

## Working with Individual Tasks

Sometimes, you may only need to test a specific part of the pipeline. To list all tasks within a DAG, use the command:

```bash
airflow tasks list mlops_pipeline
```

This command shows all individual task IDs in your pipeline, which is useful when you need to test a specific task.

To test a single task, use the `tasks test` command with the task ID and a date context, as in the example below:

```bash
airflow tasks test mlops_pipeline train_model 2023-09-15
```

This way, you can inspect and debug individual components without running the entire workflow.

-----

## Troubleshooting Common Issues

When testing ML pipelines in Airflow, you may encounter some common issues:

  * **Task dependencies not working correctly?** Double-check how you've defined the relationships between tasks. With the TaskFlow API, dependencies are created automatically when you pass the result of one task to the next.
  * **Tasks failing unexpectedly?** Try isolating and testing the problematic task. The detailed logs often reveal the source of the problem.

When troubleshooting ML pipeline issues specifically, pay close attention to domain-specific concerns like data quality issues, resource constraints (ML training often requires significant memory), and model performance thresholds that might trigger failures.

-----

## Conclusion

The Airflow CLI is an essential tool for thoroughly testing your ML pipelines. By mastering these commands, you can ensure your model retraining system functions correctly before it's deployed to production. In the next lesson, we'll discuss more advanced Airflow features for handling large datasets and optimizing resource usage.

## Discover Your Airflow Pipelines

You have just learned how to use the Airflow CLI to explore and manage your ML pipelines. Now, let's put that knowledge into practice.

Your task is to write a command in the solution.sh file that will list all the available DAGs in your Airflow environment. When you run it, you should see the mlops_pipeline DAG. This is a simple but important first step in working with Airflow from the command line.

```python
#airflow_dags.py
from datetime import datetime, timedelta
from airflow.decorators import dag, task

# Define default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
@dag(
    dag_id='mlops_pipeline',  # Unique identifier for the DAG
    description='Introduction to Airflow for ML pipelines',
    default_args=default_args,
    schedule_interval='@daily',  # Run daily
    start_date=datetime(2023, 1, 1),  # Start date (in the past)
    catchup=False,  # Don't run for past dates
    tags=['intro', 'ml'],
)
def training_pipeline():
    """    
    This DAG introduces Airflow concepts for ML workflows.
    It demonstrates task definition, dependencies, and flow control
    before we implement our actual ML pipeline in the next unit.
    """
    
    # Define tasks using the TaskFlow API
    @task(task_id="extract_data")
    def extract_data():
        """Simulate extracting data from a source."""
        print("Extracting data from source...")
        # In a real scenario, this would connect to a data source
        return {"data_extracted": True, "records": 1000}
    
    @task(task_id="transform_data")
    def transform_data(extract_result):
        """Simulate transforming the extracted data."""
        if extract_result["data_extracted"]:
            num_records = extract_result["records"]
            print(f"Transforming {num_records} records...")
            # Simulate data transformation
            return {"data_transformed": True, "features": 10}
        else:
            raise ValueError("Data extraction failed")
    
    @task(task_id="train_model")
    def train_model(transform_result):
        """Simulate training a machine learning model."""
        if transform_result["data_transformed"]:
            num_features = transform_result["features"]
            print(f"Training model with {num_features} features...")
            # Simulate model training
            return {"model_trained": True, "accuracy": 0.85}
        else:
            raise ValueError("Data transformation failed")
    
    @task(task_id="validate_model")
    def validate_model(train_result):
        """Simulate validating the model's performance."""
        if train_result["model_trained"]:
            accuracy = train_result["accuracy"]
            print(f"Validating model. Accuracy: {accuracy}")
            # Determine if model meets quality threshold
            if accuracy >= 0.8:
                return {"validation": "passed", "accuracy": accuracy}
            else:
                return {"validation": "failed", "accuracy": accuracy}
        else:
            raise ValueError("Model training failed")
    
    @task(task_id="deploy_model", trigger_rule="none_failed")
    def deploy_model(validation_result):
        """Simulate deploying the model if validation passed."""
        if validation_result["validation"] == "passed":
            print(f"Deploying model with accuracy: {validation_result['accuracy']}")
            return {"deployed": True}
        else:
            logging.warning(f"Model validation failed with accuracy: {validation_result['accuracy']}")
            return {"deployed": False}
    
    # Define the task dependencies using TaskFlow
    # This creates the DAG structure automatically based on function calls
    extract_result = extract_data()
    transform_result = transform_data(extract_result)
    train_result = train_model(transform_result)
    validation_result = validate_model(train_result)
    deploy_model(validation_result)

# Now actually enable this to be run as a DAG
training_pipeline()
```
```bash
# solution.sh
```

To list all available DAGs in your Airflow environment, you need to use the `airflow dags list` command. This command will show a list of DAGs, including the `mlops_pipeline` DAG that is defined in the `airflow_dags.py` file.

Here is the command you should write in `solution.sh`:

```bash
# solution.sh
airflow dags list
```

## Inspecting Your ML Pipeline Structure

You’ve just learned how to use the Airflow CLI to explore your available DAGs and view their details. Now, let’s take the next step and work with a specific DAG in your environment.

The solution.sh file contains a command that should show the structure of our DAG defined in airflow_dag.py. However, for some reason it appears to result in an error when it is run. Can you find the issue and fix it?

```python
from datetime import datetime, timedelta
from airflow.decorators import dag, task

# Define default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
@dag(
    dag_id='mlops_pipeline',  # Unique identifier for the DAG
    description='Introduction to Airflow for ML pipelines',
    default_args=default_args,
    schedule_interval='@daily',  # Run daily
    start_date=datetime(2023, 1, 1),  # Start date (in the past)
    catchup=False,  # Don't run for past dates
    tags=['intro', 'ml'],
)
def training_pipeline():
    """    
    This DAG introduces Airflow concepts for ML workflows.
    It demonstrates task definition, dependencies, and flow control
    before we implement our actual ML pipeline in the next unit.
    """
    
    # Define tasks using the TaskFlow API
    @task(task_id="extract_data")
    def extract_data():
        """Simulate extracting data from a source."""
        print("Extracting data from source...")
        # In a real scenario, this would connect to a data source
        return {"data_extracted": True, "records": 1000}
    
    @task(task_id="transform_data")
    def transform_data(extract_result):
        """Simulate transforming the extracted data."""
        if extract_result["data_extracted"]:
            num_records = extract_result["records"]
            print(f"Transforming {num_records} records...")
            # Simulate data transformation
            return {"data_transformed": True, "features": 10}
        else:
            raise ValueError("Data extraction failed")
    
    @task(task_id="train_model")
    def train_model(transform_result):
        """Simulate training a machine learning model."""
        if transform_result["data_transformed"]:
            num_features = transform_result["features"]
            print(f"Training model with {num_features} features...")
            # Simulate model training
            return {"model_trained": True, "accuracy": 0.85}
        else:
            raise ValueError("Data transformation failed")
    
    @task(task_id="validate_model")
    def validate_model(train_result):
        """Simulate validating the model's performance."""
        if train_result["model_trained"]:
            accuracy = train_result["accuracy"]
            print(f"Validating model. Accuracy: {accuracy}")
            # Determine if model meets quality threshold
            if accuracy >= 0.8:
                return {"validation": "passed", "accuracy": accuracy}
            else:
                return {"validation": "failed", "accuracy": accuracy}
        else:
            raise ValueError("Model training failed")
    
    @task(task_id="deploy_model", trigger_rule="none_failed")
    def deploy_model(validation_result):
        """Simulate deploying the model if validation passed."""
        if validation_result["validation"] == "passed":
            print(f"Deploying model with accuracy: {validation_result['accuracy']}")
            return {"deployed": True}
        else:
            logging.warning(f"Model validation failed with accuracy: {validation_result['accuracy']}")
            return {"deployed": False}
    
    # Define the task dependencies using TaskFlow
    # This creates the DAG structure automatically based on function calls
    extract_result = extract_data()
    transform_result = transform_data(extract_result)
    train_result = train_model(transform_result)
    validation_result = validate_model(train_result)
    deploy_model(validation_result)

# Now actually enable this to be run as a DAG
training_pipeline()

```

````bash
# solution.sh
airflow show dags ml_pipeline
```

Pemasalahan pada file `solution.sh` adalah nama DAG yang salah. Dalam file `airflow_dag.py`, `dag_id` yang didefinisikan adalah `mlops_pipeline`, bukan `ml_pipeline`.

Untuk memperbaiki masalah tersebut, Anda perlu mengganti `ml_pipeline` dengan `mlops_pipeline` di perintah `airflow show dags`.

**File `solution.sh` yang sudah diperbaiki:**

```bash
airflow dags show mlops_pipeline
```

## Test Your Full ML Pipeline

You’ve already learned how to list and inspect your Airflow DAGs using the CLI. Now, let’s move forward and try running your entire machine learning pipeline from start to finish.

Your task is to write a command in the solution.sh file that will test the entire mlops_pipeline DAG using the Airflow CLI. This is a key step in ensuring your pipeline works as expected before you schedule it for regular runs.

```python
from datetime import datetime, timedelta
from airflow.decorators import dag, task

# Define default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
@dag(
    dag_id='mlops_pipeline',  # Unique identifier for the DAG
    description='Introduction to Airflow for ML pipelines',
    default_args=default_args,
    schedule_interval='@daily',  # Run daily
    start_date=datetime(2023, 1, 1),  # Start date (in the past)
    catchup=False,  # Don't run for past dates
    tags=['intro', 'ml'],
)
def training_pipeline():
    """    
    This DAG introduces Airflow concepts for ML workflows.
    It demonstrates task definition, dependencies, and flow control
    before we implement our actual ML pipeline in the next unit.
    """
    
    # Define tasks using the TaskFlow API
    @task(task_id="extract_data")
    def extract_data():
        """Simulate extracting data from a source."""
        print("Extracting data from source...")
        # In a real scenario, this would connect to a data source
        return {"data_extracted": True, "records": 1000}
    
    @task(task_id="transform_data")
    def transform_data(extract_result):
        """Simulate transforming the extracted data."""
        if extract_result["data_extracted"]:
            num_records = extract_result["records"]
            print(f"Transforming {num_records} records...")
            # Simulate data transformation
            return {"data_transformed": True, "features": 10}
        else:
            raise ValueError("Data extraction failed")
    
    @task(task_id="train_model")
    def train_model(transform_result):
        """Simulate training a machine learning model."""
        if transform_result["data_transformed"]:
            num_features = transform_result["features"]
            print(f"Training model with {num_features} features...")
            # Simulate model training
            return {"model_trained": True, "accuracy": 0.85}
        else:
            raise ValueError("Data transformation failed")
    
    @task(task_id="validate_model")
    def validate_model(train_result):
        """Simulate validating the model's performance."""
        if train_result["model_trained"]:
            accuracy = train_result["accuracy"]
            print(f"Validating model. Accuracy: {accuracy}")
            # Determine if model meets quality threshold
            if accuracy >= 0.8:
                return {"validation": "passed", "accuracy": accuracy}
            else:
                return {"validation": "failed", "accuracy": accuracy}
        else:
            raise ValueError("Model training failed")
    
    @task(task_id="deploy_model", trigger_rule="none_failed")
    def deploy_model(validation_result):
        """Simulate deploying the model if validation passed."""
        if validation_result["validation"] == "passed":
            print(f"Deploying model with accuracy: {validation_result['accuracy']}")
            return {"deployed": True}
        else:
            logging.warning(f"Model validation failed with accuracy: {validation_result['accuracy']}")
            return {"deployed": False}
    
    # Define the task dependencies using TaskFlow
    # This creates the DAG structure automatically based on function calls
    extract_result = extract_data()
    transform_result = transform_data(extract_result)
    train_result = train_model(transform_result)
    validation_result = validate_model(train_result)
    deploy_model(validation_result)

# Now actually enable this to be run as a DAG
training_pipeline()
```

```bash
# solution.sh
# TODO: Write the command to test the entire mlops_pipeline DAG
```

Untuk menjalankan seluruh DAG dari awal hingga akhir menggunakan Airflow CLI, Anda harus menggunakan perintah `dags test` dengan ID DAG dan tanggal eksekusi. Tanggal harus berada di masa lalu agar Airflow tidak menunggu jadwal.

Berikut adalah perintah yang benar untuk file `solution.sh`:

```bash
airflow dags test mlops_pipeline 2023-01-01
```

Perintah ini akan menjalankan seluruh DAG `mlops_pipeline` secara lokal untuk tanggal yang ditentukan, tanpa melibatkan scheduler atau mencatat hasilnya ke database metadata Airflow. Ini adalah cara yang ideal untuk menguji alur kerja secara lengkap sebelum menjalankannya di lingkungan produksi.

Untuk mempelajari lebih lanjut tentang pengujian DAG Airflow secara interaktif, tonton video ini [How to easily test your Airflow DAGs with the new dag.test() function](https://www.youtube.com/watch?v=eDHQ2d1wqD4).
http://googleusercontent.com/youtube_content/4

## See All Tasks in Your Pipeline

After learning how to test your complete ML pipeline using the Airflow CLI, it's time to focus on the individual building blocks of your workflow.

Your task is to write a command in the solution.sh file that will list all the tasks defined in the mlops_pipeline DAG. This will help you see each step in your pipeline and is useful when you want to test or inspect specific tasks.

```python
from datetime import datetime, timedelta
from airflow.decorators import dag, task

# Define default arguments for the DAG
default_args = {
    'owner': 'airflow',
    'depends_on_past': False,
    'email_on_failure': False,
    'email_on_retry': False,
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
}

# Define the DAG
@dag(
    dag_id='mlops_pipeline',  # Unique identifier for the DAG
    description='Introduction to Airflow for ML pipelines',
    default_args=default_args,
    schedule_interval='@daily',  # Run daily
    start_date=datetime(2023, 1, 1),  # Start date (in the past)
    catchup=False,  # Don't run for past dates
    tags=['intro', 'ml'],
)
def training_pipeline():
    """    
    This DAG introduces Airflow concepts for ML workflows.
    It demonstrates task definition, dependencies, and flow control
    before we implement our actual ML pipeline in the next unit.
    """
    
    # Define tasks using the TaskFlow API
    @task(task_id="extract_data")
    def extract_data():
        """Simulate extracting data from a source."""
        print("Extracting data from source...")
        # In a real scenario, this would connect to a data source
        return {"data_extracted": True, "records": 1000}
    
    @task(task_id="transform_data")
    def transform_data(extract_result):
        """Simulate transforming the extracted data."""
        if extract_result["data_extracted"]:
            num_records = extract_result["records"]
            print(f"Transforming {num_records} records...")
            # Simulate data transformation
            return {"data_transformed": True, "features": 10}
        else:
            raise ValueError("Data extraction failed")
    
    @task(task_id="train_model")
    def train_model(transform_result):
        """Simulate training a machine learning model."""
        if transform_result["data_transformed"]:
            num_features = transform_result["features"]
            print(f"Training model with {num_features} features...")
            # Simulate model training
            return {"model_trained": True, "accuracy": 0.85}
        else:
            raise ValueError("Data transformation failed")
    
    @task(task_id="validate_model")
    def validate_model(train_result):
        """Simulate validating the model's performance."""
        if train_result["model_trained"]:
            accuracy = train_result["accuracy"]
            print(f"Validating model. Accuracy: {accuracy}")
            # Determine if model meets quality threshold
            if accuracy >= 0.8:
                return {"validation": "passed", "accuracy": accuracy}
            else:
                return {"validation": "failed", "accuracy": accuracy}
        else:
            raise ValueError("Model training failed")
    
    @task(task_id="deploy_model", trigger_rule="none_failed")
    def deploy_model(validation_result):
        """Simulate deploying the model if validation passed."""
        if validation_result["validation"] == "passed":
            print(f"Deploying model with accuracy: {validation_result['accuracy']}")
            return {"deployed": True}
        else:
            logging.warning(f"Model validation failed with accuracy: {validation_result['accuracy']}")
            return {"deployed": False}
    
    # Define the task dependencies using TaskFlow
    # This creates the DAG structure automatically based on function calls
    extract_result = extract_data()
    transform_result = transform_data(extract_result)
    train_result = train_model(transform_result)
    validation_result = validate_model(train_result)
    deploy_model(validation_result)

# Now actually enable this to be run as a DAG
training_pipeline()
```

```bash
# solution.sh
# TODO: Write the command to list all tasks in the mlops_pipeline DAG
```

Untuk membuat daftar semua task dalam DAG `mlops_pipeline` menggunakan Airflow CLI, Anda dapat menggunakan perintah `tasks list` diikuti dengan ID DAG.

Berikut adalah perintah yang benar untuk file `solution.sh`:

```bash
airflow tasks list mlops_pipeline
```

Perintah ini akan menampilkan daftar semua task\_id yang ada di dalam DAG yang ditentukan, memberikan gambaran yang jelas tentang struktur alur kerja Anda.

## Test an Individual Pipeline Task